<a href="https://colab.research.google.com/github/RaghavJ786/Sketcher/blob/master/Guess_the_figure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras


In [0]:
!wget 'https://raw.githubusercontent.com/zaidalyafeai/zaidalyafeai.github.io/master/sketcher/mini_classes.txt'

--2019-03-03 16:14:41--  https://raw.githubusercontent.com/zaidalyafeai/zaidalyafeai.github.io/master/sketcher/mini_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 760 [text/plain]
Saving to: ‘mini_classes.txt.2’

mini_classes.txt.2  100%[===================>]     760  --.-KB/s    in 0s      

2019-03-03 16:14:41 (103 MB/s) - ‘mini_classes.txt.2’ saved [760/760]



In [0]:
f=open("mini_classes.txt","r")
classes=f.readlines()
f.close()

In [0]:
classes = [c.replace('\n','').replace(' ','_') for c in classes]

In [0]:
!mkdir data1

mkdir: cannot create directory ‘data1’: File exists


In [0]:


import urllib.request
def download():
  base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
  for c in classes:
    cls_url = c.replace('_','%20')
    path = base + cls_url + '.npy'
    print(path)
    urllib.request.urlretrieve(path,'data1/'+c+'.npy')

In [0]:
download()

https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/drums.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/sun.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/laptop.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/anvil.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/baseball%20bat.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/ladder.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/eyeglasses.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/grapes.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/book.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/dumbbell.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/traffic%20light.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/wristwatch.npy
https://storage.googleapis.com/quickdraw_dat

In [0]:
def load_data(root, vfold_ratio=0.2, max_items_per_class= 5000):
  all_files = glob.glob(os.path.join(root, '*.npy'))
  x=np.empty([0,784])
  y=np.empty([0])
  class_names = []
  for idx, file in enumerate(all_files):
    data=np.load(file)
    data = data[0: max_items_per_class, :]
    labels = np.full(data.shape[0],idx)
    
    x=np.concatenate((x,data),axis=0)
    y=np.append(y,labels)
    
    class_name, ext =os.path.splitext(os.path.basename(file))
    class_names.append(class_name)
    
  data = None
  labels =None
  
  permutation = np.random.permutation(y.shape[0])
  x = x[permutation, :]
  y = y[permutation]

  vfold_size = int(x.shape[0]/100*(vfold_ratio*100))

  x_test = x[0:vfold_size, :]
  y_test = y[0:vfold_size]

  x_train = x[vfold_size:x.shape[0], :]
  y_train = y[vfold_size:y.shape[0]]
  return x_train, y_train, x_test, y_test, class_names

In [0]:
x_train, y_train, x_test, y_test, class_names = load_data('data1')
num_classes = len(class_names)
image_size = 28
x_train = x_train.reshape(x_train.shape[0],image_size, image_size,1).astype('float32')
x_test = x_test.reshape(x_test.shape[0],image_size,image_size,1).astype('float32')

x_train/=255.0
x_test/=255.0
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)


In [0]:
print(len(x_train))

400000


In [0]:
model = keras.Sequential()
model.add(layers.Convolution2D(16, (3,3),padding = 'same',input_shape=x_train.shape[1:],activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Convolution2D(32, (3,3),padding='same',activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Convolution2D(64,(3,3),padding='same',activation='relu'))
model.add(layers.MaxPooling2D(pool_size = (2,2)))

model.add(layers.Flatten())

model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(100,activation='softmax'))

adam =tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer =adam, metrics=['top_k_categorical_accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
__________

In [0]:
model.fit(x=x_train,y=y_train,validation_split=0.1,batch_size=256,verbose=2, epochs=5)
score=model.evaluate(x_test,y_test,verbose=0)
print('Test accuracy: {:0.2f}%' .format(score[1]*100))

Train on 360000 samples, validate on 40000 samples
Epoch 1/5
 - 389s - loss: 1.7758 - top_k_categorical_accuracy: 0.8062 - val_loss: 1.2907 - val_top_k_categorical_accuracy: 0.8878
Epoch 2/5
 - 396s - loss: 1.1594 - top_k_categorical_accuracy: 0.9019 - val_loss: 1.0943 - val_top_k_categorical_accuracy: 0.9093
Epoch 3/5
 - 412s - loss: 1.0197 - top_k_categorical_accuracy: 0.9162 - val_loss: 0.9973 - val_top_k_categorical_accuracy: 0.9180
Epoch 4/5
 - 402s - loss: 0.9411 - top_k_categorical_accuracy: 0.9246 - val_loss: 0.9441 - val_top_k_categorical_accuracy: 0.9248
Epoch 5/5
 - 397s - loss: 0.8867 - top_k_categorical_accuracy: 0.9303 - val_loss: 0.9138 - val_top_k_categorical_accuracy: 0.9268
Test accuracy: 92.52%


In [0]:
with open('class_names.txt', 'w') as file_handler:
    for item in class_names:
        file_handler.write("{}\n".format(item))

In [0]:
!pip install tensorflowjs

    100% |████████████████████████████████| 61kB 2.8MB/s 
    100% |████████████████████████████████| 61kB 5.8MB/s 
keras 2.2.2 has requirement keras-applications==1.0.4, but you'll have keras-applications 1.0.7 which is incompatible.
keras 2.2.2 has requirement keras-preprocessing==1.0.2, but you'll have keras-preprocessing 1.0.9 which is incompatible.
  Found existing installation: Keras-Applications 1.0.4
    Uninstalling Keras-Applications-1.0.4:
      Successfully uninstalled Keras-Applications-1.0.4
  Found existing installation: Keras-Preprocessing 1.0.2
    Uninstalling Keras-Preprocessing-1.0.2:
      Successfully uninstalled Keras-Preprocessing-1.0.2


In [0]:
model.save('keras.h5')

In [0]:
!mkdir model
!tensorflowjs_converter --input_format keras keras.h5 model/

mkdir: cannot create directory ‘model’: File exists
Using TensorFlow backend.


In [0]:
!cp class_names.txt model/class_names.txt

In [0]:
!zip -r model.zip model

updating: model/ (stored 0%)
updating: model/group1-shard1of1 (deflated 7%)
updating: model/model.json (deflated 83%)
updating: model/class_names.txt (deflated 41%)


In [0]:
from google.colab import files
files.download('model.zip')